In [123]:
import sqlite3
import pandas as pd

In [124]:
def create_db_conn(db_name):
    conn = None
    try:
        conn = sqlite3.connect(db_name)
        print(f"Database '{db_name}' created successfully")

    except sqlite3.Error as e:
        print(f"The error '{e}' occurred")
    
    return conn

In [125]:
conn = create_db_conn("Try")

Database 'Try' created successfully


In [126]:
class example_db():
    def __init__(self, db_name):
        try:
           conn = sqlite3.connect(db_name)
           print(f"Database '{db_name}' created successfully")
        except sqlite3.Error as e:
           print(f"The error '{e}' occurred")
           conn = None
        self.conn = conn
        self.db_name = db_name
        self.close()

    def create_table(self, table_command):
        self.connect()
        cursor = self.conn.cursor()
        try:
            cursor.execute(table_command)
            self.conn.commit()
            print("Table created successfully")
        except sqlite3.Error as e:
            print(f"The error '{e}' occurred")
        self.close()

    def execute(self, query):
        self.connect()
        cursor = self.conn.cursor()
        try:
            cursor.execute(query)
            self.conn.commit()
            print("Query executed successfully")
        except sqlite3.Error as e:
            print(f"The error '{e}' occurred")
        self.close()

    def r_query(self, query):
        self.connect()
        cursor = self.conn.cursor()
        r = None
        try:
            cursor.execute(query)
            r = cursor.fetchall()
            self.close()
            return r
        except sqlite3.Error as e:
            print(f"The error '{e}' occurred")
            self.close()
            return r
      
    def to_df(self, query, cols):
        from_db = []

        r = self.r_query(query)
        for e in r:
            r = list(e)
            from_db.append(r)

        df = pd.DataFrame(from_db, columns=cols)
        return df
    
    def close(self):
        self.conn.close()

    def connect(self):
        self.conn = sqlite3.connect(self.db_name)



In [127]:
dbA = example_db("Example")

Database 'Example' created successfully


In [128]:
paper_table = """
    CREATE TABLE IF NOT EXISTS paper (
        paper_id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        author_id TEXT NOT NULL,
        publication_date TEXT NOT NULL
    )
"""

author_table = """
    CREATE TABLE IF NOT EXISTS author (
        author_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        paper_id INT
    )
"""

In [129]:
dbA.create_table(paper_table)
dbA.create_table(author_table)

Table created successfully
Table created successfully


In [130]:
papers = """
    INSERT INTO paper (paper_id, title, author_id, publication_date) VALUES
    (111, 'Paper A', '1', '2023-01-01'),
    (222, 'Paper B', '2', '2023-02-01'),
    (333, 'Paper C', '3', '2023-03-01')
"""

authors = """
    INSERT INTO author (author_id, name, paper_id) VALUES
    (1, 'Author A', 1),
    (2, 'Author B', 2),
    (3, 'Author C', 3)
"""

In [131]:
dbA.execute(papers)
dbA.execute(authors)

Query executed successfully
Query executed successfully


In [132]:
q1 = """
SELECT * FROM author;
"""

q2 = """
SELECT * FROM paper
WHERE paper_id > 1
ORDER BY paper_id DESC;
"""

q3 = """
SELECT paper.paper_id, paper.title, author.name
FROM paper
JOIN author
ON paper.author_id = author.author_id
WHERE author.author_id > 1;
"""

In [133]:
dbA.r_query(q3)

[(222, 'Paper B', 'Author B'), (333, 'Paper C', 'Author C')]

In [134]:
dbA.to_df(q3, ["Paper ID", "Title", "Author Name"])

,Paper ID,Title,Author Name
0,222,Paper B,Author B
1,333,Paper C,Author C


In [135]:
q4 = """
UPDATE paper
SET title = "Butter Paper"
WHERE paper_id = 2;
"""

dbA.execute(q4)

Query executed successfully


In [136]:
q5 = """
DELETE FROM paper
WHERE paper_id < 100;

"""

dbA.execute(q5)

Query executed successfully


In [140]:
dbA.r_query(q2)

[(333, 'Paper C', '3', '2023-03-01'),
 (222, 'Paper B', '2', '2023-02-01'),
 (111, 'Paper A', '1', '2023-01-01')]